# Verify device with Torch

In [1]:
import torch

In [2]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('Number of CUDA Devices:', torch.cuda.device_count())
    print('CUDA Device Name:',torch.cuda.get_device_name(0))
    print('CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
    print('CUDA version:', torch.version.cuda)
else:
    print('No CUDA-GPU available from Torch perspective.')

No CUDA-GPU available from Torch perspective.


In [3]:
torch.cuda.is_available()

False

# Verify device with Tensorflow-gpu

In [4]:
import tensorflow as tf
from tensorflow.python.client import device_lib

In [5]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10275815336052550294
]


In [6]:
# is it really a cuda device?
tf.test.is_built_with_cuda()

False

# Matrix operations using Tensorflow-gpu

In [7]:
import numpy as np
import tensorflow as tf
from time import perf_counter

### Define matrix multiplication function with TF

In [8]:
def matpow(M, n):
    """
    Takes in matrix M and multiplies it by itself n-1 times.
    """
    if n < 1:
        return M
    else:
        return tf.matmul(M, matpow(M,n-1))

### Define matrices A and B, n to calculate (A^n + B^n )

In [9]:
size = 10000
A = np.random.rand(size, size).astype('float32')
B = np.random.rand(size, size).astype('float32')
n = 10

## GPU multiplication

This is pretty much all taken from: https://gist.github.com/j-min/baae1aa56e861cab9831b3722755ae6d

In [10]:
c1 = []
c2 = []

with tf.device('/gpu:0'):
    a = tf.placeholder(tf.float32, [size, size])
    b = tf.placeholder(tf.float32, [size, size])
    c1.append(matpow(a, n))
    c2.append(matpow(b, n))

with tf.device('/cpu:0'):
  sum = tf.add_n(c1)

start_time = perf_counter()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    # Run the op.
    sess.run(sum, {a:A, b:B})
stop_time = perf_counter()

print('GPU time: %g s' % (stop_time-start_time))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

## CPU multiplication

In [ ]:
c1 = []
c2 = []

with tf.device('/cpu:0'):
    a = tf.placeholder(tf.float32, [size, size])
    b = tf.placeholder(tf.float32, [size, size])
    c1.append(matpow(a, n))
    c2.append(matpow(b, n))

with tf.device('/cpu:0'):
  sum = tf.add_n(c1)

start_time = perf_counter()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    # Run the op.
    sess.run(sum, {a:A, b:B})
stop_time = perf_counter()

print('CPU time: %g s' % (stop_time-start_time))

# Numba vectorization in the GPU -- WIP

In [ ]:
import numpy as np
from time import perf_counter
from numba import vectorize

In [ ]:
@vectorize(['float32(float32, float32)'], target='cuda')
def add_vec(v1, v2):
    return v1 + v2



N=1<<12

a = np.ones(N, dtype=np.float32)
b = np.ones(N, dtype=np.float32)
c = np.empty_like(a, dtype=a.dtype)

start_time = perf_counter()
c = add_vec(a,b)
stop_time = perf_counter()

print(c)
print('Elapsed time: %g s' % (stop_time-start_time))

del a
del b
del c

In [ ]:
del a
del b
del c

In [11]:
# Testing the version of packages #
tf.__version__

'2.4.1'

SyntaxError: invalid syntax (<ipython-input-12-993baf3640af>, line 1)